In [ ]:
# packages

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import torch.optim.lr_scheduler as lr_scheduler

In [ ]:
# utilities

class FibonacciActivation(nn.Module):
    def forward(self, x):
        # Custom Fibonacci polynomial activation function (approximation)
        phi = (1 + torch.sqrt(torch.tensor(5.0))) / 2
        psi = (1 - torch.sqrt(torch.tensor(5.0))) / 2
        x = torch.clamp(x, -50, 50)  # Restrict the range of intermediate values

        # Compute the approximation of the Fibonacci activation function
        fib = (torch.pow(phi, x) - torch.pow(psi, x)) / torch.sqrt(torch.tensor(5.0))
        fib = torch.where(
            torch.isnan(fib), torch.zeros_like(fib), fib
        )  # Replace nan with zero
        return fib


class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = torch.tensor(self.data[idx, 0], dtype=torch.float32)
        t = torch.tensor(self.data[idx, 1], dtype=torch.float32)
        u = torch.tensor(self.data[idx, 2], dtype=torch.float32)
        return x, t, u


class MaxError(nn.Module):
    def forward(self, output, target):
        return torch.max(torch.abs(output - target))

In [ ]:
# neural network architecture

class FibonacciNN(nn.Module):
    def __init__(self, n, m):
        super(FibonacciNN, self).__init__()
        self.fc1 = nn.Linear(2, n * m)
        self.hidden_layer = FibonacciActivation()
        self.fc2 = nn.Linear(n * m, 1)
        self.output_layer = nn.Linear(1, 1)

    def forward(self, a):
        b = self.fc1(a)
        c = self.hidden_layer(b)
        d = self.fc2(c)
        output = self.output_layer(d)
        return output

In [ ]:
# generating trainging data

import numpy as np
from sklearn.preprocessing import MinMaxScaler


# Define the analytical solution function
def analytical_solution(x, t):
    return np.exp(x + t)


# Function to generate training data
def generate_training_data(num_points, x_min, x_max, t_min, t_max):
    x_values = np.linspace(x_min, x_max, num_points)
    t_values = np.linspace(t_min, t_max, num_points)

    # Create a grid of (x, t) pairs
    x_grid, t_grid = np.meshgrid(x_values, t_values, indexing="ij")

    # Calculate analytical solution for each pair
    u_exact = analytical_solution(x_grid, t_grid)

    # Flatten the arrays for easier handling
    x_flat = x_grid.flatten()
    t_flat = t_grid.flatten()
    u_exact_flat = u_exact.flatten()

    # Combine x, t, and u_exact into a single array
    training_data = np.column_stack((x_flat, t_flat, u_exact_flat))

    # Normalize the training data
    scaler = MinMaxScaler()
    training_data_normalized = scaler.fit_transform(training_data)

    return training_data_normalized


# Specify the number of data points and the ranges for x and t
num_points = 50
x_min, x_max = 0, 1
t_min, t_max = 0, 1

# Generate training data
training_data = generate_training_data(num_points, x_min, x_max, t_min, t_max)

In [ ]:
# variables

custom_dataset = CustomDataset(training_data)
data_loader = DataLoader(custom_dataset, batch_size=1, shuffle=True)

n = 2
m = 3
model = FibonacciNN(n, m)
criterion = MaxError()
optimizer = optim.SGD(model.parameters(), lr=0.01)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=10)

In [ ]:
# training the neural network

losses = []  # List to store the loss values
num_epochs = 20

for epoch in range(num_epochs):
    running_loss = 0.0

    for x, t, u in data_loader:
        input_combined = torch.cat((x, t), dim=-1)
        output = model(input_combined)
        loss = criterion(output, u)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(data_loader)
    losses.append(epoch_loss)
    scheduler.step(epoch_loss)

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss}")